### List contents of bronze/silver/gold mounts:

In [ ]:
print('BRONZE:')
files = dbutils.fs.ls('/mnt/stackoverflow/bronze/')

file_paths = [file.path for file in files]

for path in file_paths:
    print(path)
print('')

print('SILVER:')
files = dbutils.fs.ls('/mnt/stackoverflow/silver/')

file_paths = [file.path for file in files]

for path in file_paths:
    print(path)
print('')

print('GOLD:')
files = dbutils.fs.ls('/mnt/stackoverflow/gold/')

file_paths = [file.path for file in files]

for path in file_paths:
    print(path)

BRONZE:
dbfs:/mnt/stackoverflow/bronze/genai/
dbfs:/mnt/stackoverflow/bronze/genai.stackexchange.com.7z
dbfs:/mnt/stackoverflow/bronze/health/
dbfs:/mnt/stackoverflow/bronze/health.stackexchange.com.7z
dbfs:/mnt/stackoverflow/bronze/iot/
dbfs:/mnt/stackoverflow/bronze/iot.stackexchange.com.7z
dbfs:/mnt/stackoverflow/bronze/lifehacks.stackexchange.com.7z

SILVER:

GOLD:


### Make a list of all .xml files extracted from stackoverflow archive:

In [ ]:
from databricks_filesystem import DatabricksFilesystem

adb_fs = DatabricksFilesystem(dbutils=dbutils)

file_list = adb_fs.filesystem_list(filesystem_path="/mnt/stackoverflow/bronze/")

xml_files = []

for file in file_list:
    if '.xml' in file:
        xml_files.append(file)

### Transform and Load
1. Make transformations:
- remove leading '_' from column names
- add 'Category' column (with fixed value)
- Clean up "free text" columns (i.e., AboutMe, Body, etc.)

2. Then write df's into silver layer.

In [ ]:
from bs4 import BeautifulSoup
from pyspark.sql.functions import lit, udf, col, substring
from pyspark.sql.types import StringType, VarcharType
import html
import re

# Function to clean HTML content
def clean_html_column(text):
    if text is not None:
        return ' '.join(html.unescape(text).split())
    return text
# Register UDF
clean_html_udf = udf(clean_html_column, StringType())

# Compile a regex pattern to convert CamelCase to snake_case
pattern = re.compile(r"(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])")

# Function to transform column name
def transform_column_name(col_name):
    # Remove leading underscore, if present
    if col_name.startswith('_'):
        col_name = col_name[1:]
    
    # Convert CamelCase to snake_case
    col_name = pattern.sub('_', col_name).lower()
    
    return col_name

for file in xml_files:
    print('')
    print(f"STARTING TRANSFORMATION FOR {file}")
    df = spark.read.option("rowTag", 'row').format("xml").load(file) #str(file.split('/')[-1].split('.')[0].lower())
    # remove leading '_' from column names
    for c in df.columns:
        new_col_name = transform_column_name(c)
        df = df.withColumnRenamed(c, new_col_name)
    print(df.schema)
    # add 'Category' column (with fixed value)
    df = df.withColumn('category', lit(str(file.split('/')[-2].lower())))
    # clean up free text columns (AboutMe, Text, etc.)
    columns_to_check = ['about_me', 'text', 'body']
    columns_present = [c for c in columns_to_check if c in df.columns]
    if columns_present:
        for c in columns_present:
            print(f"col {c} exists. Cleaning {c}")
            df = df.withColumn(c, substring(col(c), 1, 255))
            df = df.withColumn(c, clean_html_udf(df[c]))
            df = df.withColumn(c, df[c].cast(VarcharType(255)))
            print(f"col {c} cleaned.")
    else:
        print("No matching columns found.")
    dest = f'/mnt/stackoverflow/silver/{file.split("/")[-2]}/{file.split("/")[-1].split(".")[0].lower()}/' #/dbfs
    df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(dest)
    print(f'TRANSFORMATION COMPLETED FOR {file}. FILE WRITTEN TO {dest}')
    print('')


STARTING TRANSFORMATION FOR dbfs:/mnt/stackoverflow/bronze/genai/Badges.xml
StructType([StructField('class', LongType(), True), StructField('date', TimestampType(), True), StructField('id', LongType(), True), StructField('name', StringType(), True), StructField('tag_based', BooleanType(), True), StructField('user_id', LongType(), True)])
No matching columns found.
TRANSFORMATION COMPLETED FOR dbfs:/mnt/stackoverflow/bronze/genai/Badges.xml. FILE WRITTEN TO /mnt/stackoverflow/silver/genai/badges/


STARTING TRANSFORMATION FOR dbfs:/mnt/stackoverflow/bronze/genai/Comments.xml
StructType([StructField('creation_date', TimestampType(), True), StructField('id', LongType(), True), StructField('post_id', LongType(), True), StructField('score', LongType(), True), StructField('text', StringType(), True), StructField('user_display_name', StringType(), True), StructField('user_id', LongType(), True)])
col text exists. Cleaning text
col text cleaned.
TRANSFORMATION COMPLETED FOR dbfs:/mnt/stackove